In [1]:
# ultralytics

In [2]:
import cv2
import mysql.connector
from datetime import datetime
from deepface import DeepFace
import os
import pickle
import concurrent.futures

try:
    mydb = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="e@123321xzX",
        port=3306,
        database="attendance"
    )
except mysql.connector.Error as err:
    print(f"Database connection error: {err}")
    exit(1)  # Exit with an error code

In [3]:
def compute_and_save_encodings():
    def process_folder(student_folder):
        student_id = os.path.basename(student_folder)
        image_files = [os.path.join(student_folder, file) for file in os.listdir(student_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
        encodings = []
        for img_path in image_files:
            img = cv2.imread(img_path)
            encoding = DeepFace.represent(img, model_name='VGG-Face', enforce_detection=False)
            if encoding:
                encodings.append((img_path, encoding[0]['embedding']))
        return encodings

    db_folder = 'db'
    student_folders = [os.path.join(db_folder, folder) for folder in os.listdir(db_folder) if os.path.isdir(os.path.join(db_folder, folder))]

    results = []
    for student_folder in student_folders:
        results.extend(process_folder(student_folder))

    # Save computed embeddings and corresponding file paths
    with open('known_encodings.pkl', 'wb') as f:
        pickle.dump(results, f)

# Check if saved embeddings exist, if not compute and save them
if not os.path.exists('known_encodings.pkl'):
    compute_and_save_encodings()

# Load saved embeddings and file paths
with open('known_encodings.pkl', 'rb') as f:
    results = pickle.load(f)


def mark_attendance(student_id, course_id, session_id):
    """Marks attendance for a recognized student in the database."""
    try:
        cursor = mydb.cursor()
        # Fetch student's name from the database based on the student_id
        sql = "SELECT fullname FROM student_details WHERE student_id = %s"
        val = (student_id,)
        cursor.execute(sql, val)
        result = cursor.fetchone()
        if result:
            student_name = result[0]
            attendance_status = f"Attendance taken for student {student_name}"
        else:
            attendance_status = "Attendance not taken"
        # Check if the student has already been marked present for the given course and session
        sql = "SELECT * FROM attendance WHERE student_id = %s AND course_id = %s AND session_id = %s"
        val = (student_id, course_id, session_id)
        cursor.execute(sql, val)
        result = cursor.fetchone()
        if result:
            print(f"Student with ID {student_id} has already been marked present for course {course_id} and session {session_id}. Skipping insertion.")
            return
        else:
            sql = "INSERT INTO attendance (course_id, session_id, timestamp, status, student_id) VALUES (%s, %s, %s, %s, %s)"
            val = (course_id, session_id, datetime.now(), "Present", student_id)
            cursor.execute(sql, val)
            mydb.commit()
            cursor.close()
            print(f"Attendance marked for student with ID {student_id} for course {course_id} and session {session_id}.")
    except mysql.connector.Error as err:
        print(f"Database error: {err}")


# Extract known face encodings and corresponding file paths
known_encodings = [result[1] for result in results]
file_paths = [result[0] for result in results]
confidence_threshold = 0.6

In [ ]:
# Process frames
cap = cv2.VideoCapture(0)

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    success, img = cap.read()
    if not success:
        print("Error reading frame from camera.")
        break

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = img[y:y+h, x:x+w]

        face_encoding = DeepFace.represent(face_img, model_name='VGG-Face', enforce_detection=False)

        if face_encoding:
            face_encoding = face_encoding[0]['embedding']

            best_match_id = None
            min_distance = float('inf')

            for j, known_encoding in enumerate(known_encodings):
                if len(known_encoding) == len(face_encoding):
                    result = DeepFace.verify(known_encoding, face_encoding, model_name='VGG-Face', silent=True)
                    if result['verified'] and result['distance'] < min_distance:
                        if result['distance'] > confidence_threshold:  # Check against confidence threshold
                            best_match_id = os.path.splitext(os.path.basename(file_paths[j]))[0]
                            min_distance = result['distance']

            # Fetch student's name from the database based on student ID
            student_name = None
            if best_match_id:
                cursor = mydb.cursor()
                sql = "SELECT fullname FROM student_details WHERE student_id = %s"
                val = (best_match_id,)
                cursor.execute(sql, val)
                result = cursor.fetchone()
                if result:
                    student_name = result[0]

            # Check if the student is recognized and mark attendance accordingly
            if best_match_id:
                mark_attendance(best_match_id, 3, 3)
                if student_name:
                    attendance_status = f"Attendance taken for student {student_name}"
                else:
                    attendance_status = "Attendance taken for student"

                # Save the successful recognition image to the student's folder
                student_folder = os.path.join('db', best_match_id)
                if not os.path.exists(student_folder):
                    os.makedirs(student_folder)
                cv2.imwrite(os.path.join(student_folder, f"{datetime.now().strftime('%Y%m%d%H%M%S')}.jpg"), face_img)
            else:
                attendance_status = "Attendance not taken"

            # Draw rectangle and label with attendance status and student's name
            color = (0, 255, 0) if best_match_id else (0, 0, 255)
            cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
            cv2.putText(img, attendance_status, (x, y-10), cv2.FONT_ITALIC, 0.9, color, 2)
            if best_match_id and student_name:
                cv2.putText(img, student_name, (x, y+h+20), cv2.FONT_ITALIC, 0.9, color, 2)

    cv2.imshow("Faces with Rectangles", img)

    if cv2.waitKey(1) == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Student with ID 6 has already been marked present for course 3 and session 3. Skipping insertion.
Student with ID 3 has already been marked present for course 3 and session 3. Skipping insertion.
Student with ID 3 has already been marked present for course 3 and session 3. Skipping insertion.
Student with ID 3 has already been marked present for course 3 and session 3. Skipping insertion.
Student with ID 3 has already been marked present for course 3 and session 3. Skipping insertion.
Student with ID 3 has already been marked present for course 3 and session 3. Skipping insertion.
Student with ID 3 has already been marked present for course 3 and session 3. Skipping insertion.
Student with ID 3 has already been marked present for course 3 and session 3. Skipping insertion.
Student with ID 3 has already been marked present for course 3 and session 3. Skipping insertion.
Student with ID 6 has already been marked present for course 3 and session 3. Skipping insertion.
Student with ID 6 ha